Origin
  [OriginID] Integer not null primary key
  [Origin] Text not null
Destination
  [DestinationID] integer not null Primary key
  [Destination] Text not null
Trips
  [TripID] INTEGER NOT NULL PRIMARY KEY
  [pickup_datetime] TEXT NOT NULL
  [trip_distance] FLOAT NOT NULL
  [trip_duration] TEXT NOT NULL
  [OriginID] INTEGER NOT NULL REFERENCES Origin(OriginID)
  [DestinationID] INTEGER NOT NULL REFERENCES Destination(DestinationID))

In [3]:
### Utility Functions
from IPython.display import display, HTML
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows


In [2]:
def create_origin_table(data_filename, normalized_database_filename):
    conn = create_connection(normalized_database_filename)
    
    create_table_origin = '''CREATE TABLE IF NOT EXISTS Origin ( 
                         [OriginID] Integer not null primary key,
                         [Origin] Text not null);
                         ''' 
    
    row_count = 0
    origins = []
    with open(data_filename) as file:
        
        for line in file:
            if row_count ==0:
                columns = (line.strip().split(","))
                row_count+=1
            else:
                origins.append(line.strip().split(",")[1].strip())
                row_count+=1
        
        origins_unique = sorted(set(origins))
        origins_final = list(set(zip(range(1,len(origins_unique)+1),origins_unique)))
        origins_final=sorted(origins_final)
        
    with conn:
        create_table(conn, create_table_origin)
        insert_Origin(conn,origins_final)
        
def insert_Origin(conn,values):
        
        sql_insert_origin = """INSERT INTO Origin(OriginID,Origin)
                                VALUES(?,?)"""
        curr = conn.cursor()
        curr.executemany(sql_insert_origin,values)
        return curr.lastrowid

In [3]:
def create_destination_table(data_filename, normalized_database_filename):
    conn = create_connection(normalized_database_filename)
    
    create_table_destinations = '''CREATE TABLE IF NOT EXISTS Destination ( 
                                 [DestinationID] Integer not null primary key,
                                 [Destination] Text not null);
                         ''' 
    
    row_count = 0
    destinations = []

    with open(data_filename) as file:
        
        for line in file:
            if row_count ==0:
                columns = (line.strip().split(","))
                row_count+=1
            else:
                destinations.append(line.strip().split(",")[2].strip())
                row_count+=1
        
        destinations_unique = sorted(set(destinations))
        destinations_final = list(set(zip(range(1,len(destinations_unique)+1),destinations_unique)))
        destinations_final=sorted(destinations_final)
        
    with conn:
        create_table(conn, create_table_destinations)
        insert_destinations(conn,destinations_final)
        
def insert_destinations(conn,values):
        
        sql_insert_destinations = """INSERT INTO Destination(DestinationID,Destination)
                                VALUES(?,?)"""
        curr = conn.cursor()
        curr.executemany(sql_insert_destinations,values)
        return curr.lastrowid

In [4]:
def origin_to_originid_dictionary(normalized_database_filename,conn):
    
    
    # YOUR CODE HERE
    origins = execute_sql_statement("SELECT OriginID,Origin FROM Origin",conn)
    origin_dict = dict()
    for item in origins:
        origin_dict[item[1]] = item[0]
        
    return origin_dict
        

In [5]:
def destination_to_destinationid_dictionary(normalized_database_filename,conn):
    
    
    # YOUR CODE HERE
    destination = execute_sql_statement("SELECT DestinationID,Destination FROM Destination",conn)
    destination_dict = dict()
    for item in destination:
        destination_dict[item[1]] = item[0]
        
    return destination_dict

In [6]:
def create_trips_table(data_filename, normalized_database_filename):
    conn = create_connection(normalized_database_filename)
    
    create_table_trips = '''CREATE TABLE IF NOT EXISTS Trips( 
                                 [TripID] INTEGER NOT NULL PRIMARY KEY,
                                 [pickup_datetime] TEXT NOT NULL,
                                 [trip_distance] FLOAT NOT NULL,
                                 [trip_duration] TEXT NOT NULL,
                                 [OriginID] INTEGER NOT NULL REFERENCES Origin(OriginID),
                                 [DestinationID] INTEGER NOT NULL REFERENCES Destination(DestinationID));
                         ''' 
    
    row_count = 0
    trips = []
    destination_dict = destination_to_destinationid_dictionary(normalized_database_filename,conn)
    origin_dict = origin_to_originid_dictionary(normalized_database_filename,conn)
    with open(data_filename) as file:
        
        for line in file:
            if row_count ==0:
                columns = (line.strip().split(","))
                row_count+=1
            else:
                a = line.strip().split(',')
                values = (a[3],a[4],a[5],origin_dict[a[1]],destination_dict[a[2]])
                trips.append(values)
                row_count+=1
        
        
    with conn:
        create_table(conn, create_table_trips)
        insert_Trip(conn,trips)
        

def insert_Trip(conn, values):
    sql = '''INSERT INTO Trips(pickup_datetime, trip_distance,trip_duration,OriginID,DestinationID)
          VALUES(?,?,?,?,?) '''
    cur = conn.cursor()
    cur.executemany(sql, values)
    return cur.lastrowid

In [7]:
data_filename = "uber_nyc_data.csv"# sample data
normalized_database_filename = 'normalized_final.db'
create_origin_table(data_filename, normalized_database_filename)
create_destination_table(data_filename, normalized_database_filename)
create_trips_table(data_filename, normalized_database_filename)

In [46]:
# not handled missing values in destination and origin

In [8]:
def ConvertDurationToMinutes(time_str):
    mins_split = str(time_str).split(':')
    #if (len(mins_split)!=3):
        #print("nope")
        #print(mins_split)
    
    val = int(mins_split[0])*60 + int(mins_split[1]) + int(mins_split[2])/60.0
    return val

In [14]:
import datetime
def est_revenue(arr):
    base_fare = 2.55
    per_minute = 0.35
    per_mile = 1.75
    min_fare = 8
    rev = base_fare + arr[0] * per_minute + arr[1] * per_mile
    return rev if rev > min_fare else min_fare                
def convertTripDuration(non_normalized_db_filename):
    """
    Function that takes a string in the format yyyy-mm-dd hh:mm:ss, and
    returns the same as a datetime object.
    """
    conn = create_connection(non_normalized_db_filename)
     
    min_fare = 8    
    sql_statement_dates = "SELECT * from Trips"
    df = pd.read_sql_query(sql_statement_dates, conn)
    df['pickup_dt'] = df['pickup_datetime'].astype('datetime64[ns]')
    df['date'] = df['pickup_dt'].dt.date
    df['year'] = df['pickup_dt'].dt.year
    df['month'] = df['pickup_dt'].dt.month
    df['day'] = df['pickup_dt'].dt.day
    df['hour'] = df['pickup_dt'].dt.hour 
    df['weekday'] = df['pickup_dt'].dt.dayofweek
    
    
    #sql_statement = "SELECT TripID,trip_duration,trip_distance,OriginID,DestinationID from Trips"
    #df = pd.read_sql_query(sql_statement, conn)
    
#     df = df[df['trip_distance'].notna()]
#     df = df[df['trip_duration'].notna()]
    df = df.dropna(subset=['trip_distance','trip_duration']) 
    df['trip_duration'] = df['trip_duration'].replace('NULL','0:0:0')
    df['duration_in_mins']=df['trip_duration'].apply(ConvertDurationToMinutes)
    
    
    df_DistDur = df.groupby(['OriginID', 'DestinationID'])[['trip_distance', 'duration_in_mins']].mean()
    df38 = df[df.trip_duration.isnull() & df.trip_distance.isnull()]
    for i in df38.index:
        orig = df.loc[i, 'OriginID']
        dest = df.loc[i, 'DestinationID']
        df.loc[i, 'trip_distance'] = df_DistDur.loc[orig, dest].trip_distance
        df.loc[i, 'duration_in_mins'] = df_DistDur.loc[orig, dest].duration_in_mins


    print(df.shape)
    df = df[df['duration_in_mins']!=0.0]
    df['trip_mph_avg'] = df['trip_distance']/(df['duration_in_mins']/60.0)
    df_new = df[['duration_in_mins', 'trip_distance']].values
    
    df['est_revenue'] = pd.Series(map(lambda x: est_revenue(x), df_new)) 
    df.loc[df.est_revenue < 8, 'est_revenue'] = min_fare
    print(df.info())    
    
    return df[(df['pickup_dt'] != datetime.date(2015, 9, 1)) & (df['duration_in_mins'] <= 960)]

In [15]:
df=convertTripDuration("normalized_final.db")
df_final = df.drop(['pickup_datetime','trip_duration'],axis=1)

(30925738, 14)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30900834 entries, 0 to 30925737
Data columns (total 16 columns):
 #   Column            Dtype         
---  ------            -----         
 0   TripID            int64         
 1   pickup_datetime   object        
 2   trip_distance     object        
 3   trip_duration     object        
 4   OriginID          int64         
 5   DestinationID     int64         
 6   pickup_dt         datetime64[ns]
 7   date              object        
 8   year              int64         
 9   month             int64         
 10  day               int64         
 11  hour              int64         
 12  weekday           int64         
 13  duration_in_mins  float64       
 14  trip_mph_avg      object        
 15  est_revenue       float64       
dtypes: datetime64[ns](1), float64(2), int64(8), object(5)
memory usage: 3.9+ GB
None


In [16]:
df_final.head(5)

,TripID,trip_distance,OriginID,DestinationID,pickup_dt,date,year,month,day,hour,weekday,duration_in_mins,trip_mph_avg,est_revenue
0,1,4.25,26,23,2014-09-01 09:00:00,2014-09-01,2014,9,1,9,0,15.183333,16.7947,15.301667
1,2,10.17,25,7,2014-09-01 18:00:00,2014-09-01,2014,9,1,18,0,34.083333,17.9032,32.276667
2,3,4.02,3,11,2014-09-01 17:00:00,2014-09-01,2014,9,1,17,0,17.100000,14.1053,15.570000
3,4,1.46,14,17,2014-09-01 13:00:00,2014-09-01,2014,9,1,13,0,6.533333,13.4082,8.000000
4,5,8.31,10,2,2014-09-01 14:00:00,2014-09-01,2014,9,1,14,0,26.283333,18.9702,26.291667


In [ ]:
import matplotlib.mlab as mlab
from matplotlib import pyplot as plt
import plotly.plotly as py
from matplotlib.ticker import FuncFormatter #Call formatter function to format tick values
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox) #Create image box
from matplotlib._png import read_png #Load png file
from matplotlib.patches import Ellipse #Draw ellipse

In [ ]:
def thousands_comma(x, pos):
    return '{:,.0f}'.format(x) 

def thousands_format(x, pos):
    return '{:.0f}{}'.format(x * 1e-3, 'K')

def millions_format(x, pos):
    return '{:.1f}{}'.format(x * 1e-6, 'M') 

def millions_currency(x, pos):
    return '{}{:.0f}{}'.format('$', x * 1e-6, 'M')

def annotate_labels(ax, labels_list, **kwargs):
    (y_bottom, y_top) = ax.get_ylim()
    y_height = y_top - y_bottom
    
    rects = ax.patches

    for rect, label in zip(rects, labels_list):
        height = rect.get_height()
        p_height = (height / y_height) 
        label_position = height + (y_height * 0.01)
        ax.text(rect.get_x() + rect.get_width()/2., label_position, label, kwargs)
    return None

In [ ]:
byDate = df_final.groupby('pickup_dt')['TripID'].count() #365 complete entries

fig = plt.figure()
 
ax = byDate.plot(figsize = (16, 8), fontsize = 12, ylim = (10000, 170000), color = 'navy')
 
formatter = FuncFormatter(thousands_format)
ax.yaxis.set_major_formatter(formatter)
ax.set_axis_bgcolor('#F9F9F9')
plt.title('Total Trips per Day with Annotation of Some Major Events and Holidays', fontsize= 20, color='navy')
plt.tick_params(labelsize=14)
plt.xlabel('')
 
img1 = read_png('icons/Snow-48.png')
imagebox = OffsetImage(img1, zoom=0.6)
xy = ['2015-01-27', 25000] 
ab = AnnotationBbox(imagebox, xy, xybox=(22., 10.), xycoords='data', boxcoords='offset points', pad=0.1, frameon=False) 
ax.add_artist(ab)
 
img2 = read_png('icons/Thanksgiving-48.png')
imagebox = OffsetImage(img2, zoom=0.6) 
xy = ['2014-11-27', 40000] 
ab = AnnotationBbox(imagebox, xy, xybox=(5., -5.), xycoords='data', boxcoords='offset points', pad=0.1, frameon=False)
ax.add_artist(ab)
 
img2 = read_png('icons/Christmas Tree-48.png')
imagebox = OffsetImage(img2, zoom=0.6) 
xy = ['2014-12-25', 25000] 
ab = AnnotationBbox(imagebox, xy, xybox=(10., 5.), xycoords='data', boxcoords='offset points', pad=0.1, frameon=False)
ax.add_artist(ab)
 
img3 = read_png('icons/Leave-48.png')
imagebox = OffsetImage(img3, zoom=0.6) 
xy = ['2015-05-25', 60000] 
ab = AnnotationBbox(imagebox, xy, xybox=(1., 5.), xycoords='data', boxcoords='offset points', pad=0.1, frameon=False)
ax.add_artist(ab)
 
img4 = read_png('icons/Rainbow-48.png')
imagebox = OffsetImage(img4, zoom=0.6) 
xy = ['2015-06-27', 150000] 
ab = AnnotationBbox(imagebox, xy, xybox=(18., 2.), xycoords='data', boxcoords='offset points', pad=0.1, frameon=False)
ax.add_artist(ab)
 
img5 = read_png('icons/Leave-48.png')
imagebox = OffsetImage(img5, zoom=0.6) 
xy = ['2015-07-03', 70000] 
ab = AnnotationBbox(imagebox, xy, xybox=(5., -10.), xycoords='data', boxcoords='offset points', pad=0.1, frameon=False)
ax.add_artist(ab)
 
plt.show()


## Bar Graph  for monthly base revenue in nyc:

In [ ]:
weekday_labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
month_labels = ['Sep', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug']

In [ ]:
import seaborn as sns

byDateRev = df_final.groupby('month')['est_revenue'].sum()
sumRev = byDateRev.sum()
print('${:,.0f}M'.format(sumRev * 1e-6), "Base Revenue   Uber's Base Gross Margin:", '${:,.0f}'.format(sumRev * .25))
len(df_final2) * 27 * .25
months_seq = [9,10,11,12,1,2,3,4,5,6,7,8]
formatter = FuncFormatter(millions_currency)
 
plt.figure(figsize = (14, 7))
ax = sns.barplot(byDateRev.index, byDateRev.values, order = months_seq)
ax.set_xticklabels(month_labels)
ax.yaxis.set_major_formatter(formatter)
 
font = {'color': 'navy', 'size': 15}
plt.text(0, 55000000, 'Total Base Revenue for the Period: ${:,.0f} Million'.format(sumRev * 1e-6), fontdict = font) 
plt.xlabel('')
plt.ylabel('Revenue (Millions)', fontsize = 15)
plt.tick_params(labelsize=13)
plt.title('Estimated Monthly Base Revenue from Sep-2014 to Aug-2015', fontsize = 17, color='navy')
 
plt.show()


In [ ]:
dict_growth = {}
for i in range(len(months_seq)):
    mo = months_seq[i]
    rev = byDateRev[mo]
    if mo == 9:
        growth_pct = 0
    elif mo == 1:
        growth_pct = ((rev/byDateRev[12]) - 1) * 100
    else:
        growth_pct = ((rev/byDateRev[mo - 1]) - 1) * 100
    print(month_labels[i], 'Revenue =', '${:,.0f}'.format(rev), 'Growth % = ', '{:.1f}'.format(growth_pct))
    dict_growth[month_labels[i]] = growth_pct

In [ ]:
df_plt1 = pd.DataFrame(pd.Series(dict_growth), index = month_labels, columns = ['growth_pct'])

labels_list = df_plt1.growth_pct.values.round(1)
kwargs = {'fontsize': 12, 'ha':'center', 'va': 'bottom', 'weight': 'bold', 'color': 'navy'}

ax = df_plt1.plot(kind = 'bar', figsize = (15,7), fontsize = 14, rot = 0, legend=False)#df_plt2[1:] to exclude Sep
ax.set_axis_bgcolor('#F9F9F9')
ax.get_yaxis().set_ticks([])
annotate_labels(ax, labels_list, **kwargs)

plt.axhline(0, color='yellow')
plt.title('Month Over Month Percentage Growth of Base Revenue From September 2014', 
          fontsize = 17, color='navy')
plt.ylabel('Growth %', fontsize = 15, weight='bold', color='navy')
plt.ylim(-2, 22)
plt.tick_params(labelsize=15)

plt.show()

In [ ]:
print('Cummulative % Growth Over Period:', df_plt1.growth_pct.sum())

df_plt1.cumsum().plot(color = 'navy', marker = 'D', legend = False, figsize=(16, 6))
plt.title('Cummulative Revenue Growth Percentage from September 2014', fontsize=17, weight='bold', color='navy')
plt.tick_params(labelsize=14)
plt.show()

In [ ]:
fig = plt.figure(figsize = (16,12))
formatter = FuncFormatter(thousands_format)


plt.subplot(2,2,1)
ax1 = df_final[(df_final.weekday < 5) & (df_final.trip_distance >= 5)].\
groupby('hour')['trip_distance'].count().plot(kind='bar', rot = 0)
ax1.yaxis.set_major_formatter(formatter)
ellipse = Ellipse(xy=(5, 150000), width=4, height=200000, edgecolor='red', fc='None', lw=1.5)
ax1.add_patch(ellipse)
plt.xlabel('Hour', fontsize=14, weight='bold', color='navy')
plt.ylabel('Demand (number of trips)', fontsize=14, weight='bold', color='navy')
plt.ylim(0, 1400000)
plt.title('Weekday, 5 miles or more', fontsize=14, weight='bold', color='navy')

plt.subplot(2,2,2)
ax2 = df_final[(df_final.weekday < 5) & (df_final.trip_distance < 5)].\
groupby('hour')['trip_distance'].count().plot(kind='bar', rot = 0)
ax2.yaxis.set_major_formatter(formatter)
ellipse = Ellipse(xy=(5, 150000), width=4, height=200000, edgecolor='red', fc='None', lw=1.5)
ax2.add_patch(ellipse)
plt.xlabel('Hour', fontsize=14, weight='bold', color='navy')
plt.title('Weekday, less than 5 miles', fontsize=14, weight='bold', color='navy')

plt.subplot(2,2,3)
ax3 = df_final[(df_viz.weekday >= 5) & (df_final.trip_distance >= 5)].\
groupby('hour')['trip_distance'].count().plot(kind='bar', rot = 0)
ax3.yaxis.set_major_formatter(formatter)
ellipse = Ellipse(xy=(6.5, 50000), width=4, height=80000, edgecolor='red', fc='None', lw=1.5)
ax3.add_patch(ellipse)
plt.xlabel('Hour', fontsize=14, weight='bold', color='navy')
plt.ylabel('Demand (number of trips)', fontsize=14, weight='bold', color='navy')
plt.ylim(0, 500000)
plt.title('Weekend, 5 miles or more', fontsize=14, weight='bold', color='navy')

plt.subplot(2,2,4)
ax4 = df_final[(df_final.weekday >= 5) & (df_final.trip_distance < 5)].\
groupby('hour')['trip_distance'].count().plot(kind='bar', rot = 0)
ax4.yaxis.set_major_formatter(formatter)
ellipse = Ellipse(xy=(6.5, 50000), width=4, height=80000, edgecolor='red', fc='None', lw=1.5)
ax4.add_patch(ellipse)
plt.xlabel('Hour', fontsize=14, weight='bold', color='navy')
plt.title('Weekend, less than 5 miles', fontsize=14, weight='bold', color='navy')
fig.subplots_adjust(hspace=0.4)

plt.show()

In [ ]:
fig = plt.figure(figsize = (16,8))

plt.subplot(1,2,1)
ax1 = df_final.OriginID.value_counts(ascending = True).plot(kind = 'barh', color = 'navy')
ax1.set_xticklabels(['0', '1M', '2M', '3M', '4M', '5M', '6M'])
plt.tick_params(labelsize=12)
plt.xlabel('Number of Trips per Origin Locations', fontsize = 16, color='navy')

plt.subplot(1,2,2) 
ax2 = df_final.DestinationID.value_counts(ascending = True).plot(kind = 'barh')
ax2.set_xticklabels(['0', '1M', '2M', '3M', '4M', '5M', '6M'])
plt.tick_params(labelsize=12)
plt.xlabel('Number of Trips per Destination Locations', fontsize = 16, color='navy')

plt.show()

In [ ]:
df_plt7 = df_final[df_final.weekday < 5].groupby('hour')['trip_mph_avg'].median()

plt.figure(figsize =(14, 6))
kwargs = {'fontsize': 12, 'ha':'center', 'va': 'top', 'color': 'navy', 'weight': 'bold'}

ax = df_plt7.plot(marker = 'o', color = 'navy')

for x, y in zip(df_plt7.index, df_plt7.values):
    ax.annotate('{:.0f}'.format(y), xy=(x, y), xytext= (0, 24), textcoords='offset points', **kwargs)

ax.set_axis_bgcolor('#F9F9F9')
ax.get_yaxis().set_ticks([]) 
plt.fill([7,9,9,7], [0,0,30,30], 'cyan', alpha=0.2)
plt.fill([16,18,18,16], [0,0,30,30], 'cyan', alpha=0.2)
plt.xticks(range(24))
plt.xlabel('Hour', fontsize=14)
plt.ylabel('Trip Average Speed', fontsize=14)
plt.ylim(5, 30)
plt.xlim(-0.5, 23.5)
plt.tick_params(labelsize=14)
plt.title('Weekday Average Speed per Hour of the Day - Highlight for Peak Traffic', fontsize = 16, color='navy')
plt.show()